In [ ]:
Pkg.add("Jacobi")
Pkg.add("FastGaussQuadrature")
Pkg.add("Plots")
Pkg.update()

In [2]:
using FastGaussQuadrature
using Jacobi

f_0 = 1
H = 1
nodePoints=1000

function b_n(n::Int64)
    numerator = -n*(n+1)
    denominator = (n+2)*(n+3)
    return numerator/denominator
end

function M_ij(size::Int64)
    fullMatrix = zeros(size,size)
    for i=0:size-1
        for j=0:size-1
            if i==j
                fullMatrix[i+1, j+1] = 2/(2*i+1) + (b_n(i)*b_n(i))*(2/(2*i+5))
            elseif i==j+2
                fullMatrix[i+1, j+1] = b_n(j)*(2/(2*j+5))
            elseif i==j-2
                fullMatrix[i+1, j+1] = b_n(i)*(2/(2*j+5))
            end
        end
    end
    return fullMatrix/2
end

function B_ij(size::Int64)
    fullMatrix = zeros(size,size)
    for i=0:size-1
        for j=0:size-1
            if i==j
                fullMatrix[i+1,j+1]=2/(2*i+1)
            elseif i==j-2
                fullMatrix[i+1,j+1]=b_n(i)*2/(2*i+5)
            end
        end
    end
    return fullMatrix/2
end

function L_ij(size::Int64, N::Function)
    fullArray = zeros(size,size)
    values, weights = gausslegendre(nodePoints)
    for i=0:size-1
        for j=0:size-1
            sumTotal = 0
            for k=0:nodePoints-1
                dp_i = 2.*(djacobi(values[k+1],i,0,0)+b_n(i)*djacobi(values[k+1],i+2,0,0))
                dp_j = 2.*(djacobi(values[k+1],j,0,0)+b_n(j)*djacobi(values[k+1],j+2,0,0))
                newPiece = weights[k+1]*(f_0*f_0/(H*H*N((values[k+1]+1.)/2.)*N((values[k+1]+1.)/2.)))*dp_i*dp_j
                sumTotal = sumTotal + newPiece
            end
            fullArray[i+1,j+1] = .5*sumTotal
        end
    end
    return fullArray
end

L_ij (generic function with 1 method)

In [3]:
function dq_g_rhs(size::Int64, dq_bar::Function)
    fullArray = zeros(size)
    values, weights = gausslegendre(nodePoints)
    for i =0:size-1
        sumTotal = 0
        for k=0:nodePoints-1
            p_n = jacobi(values[k+1], i,0,0) + b_n(i)*jacobi(values[k+1],i+2,0,0)
            sumTotal = sumTotal + weights[k+1]*p_n*dq_bar((values[k+1]+1.)/2.)
        end
        fullArray[i+1] = .5*sumTotal
    end
    return fullArray
end

function dq_galerkin_coefficients(size::Int64, dq_bar::Function)
    u_rhs = dq_g_rhs(size, dq_bar)
    return u_rhs
end

function dq_galerkin(size::Int64, coefficients, value::Float64)
    sumTotal = 0
    for i=0:size-1
        sumTotal = sumTotal + coefficients[i+1]*jacobi(2.*value-1.,i,0,0)
    end
    return sumTotal
end

function p_psi_function(size::Int64, value::Float64)
    fullArray = zeros(size)
    for i=0:size-1
        fullArray[i+1] = jacobi(2.*value-1.,i,0,0)+b_n(i)*jacobi(2.*value-1.,i+2,0,0)
    end
    return fullArray
end

function p_q_function(size::Int64, value::Float64)
    fullArray = zeros(size)
    for i=0:size-1
        fullArray[i+1] = jacobi(2.*value-1.,i,0,0)
    end
    return fullArray
end

function dy_vartheta_bar(N::Function, du_bar::Function, value::Float64)
    return -(f_0*f_0/(H*H*N(value)*N(value)))*du_bar(value)
end

function dy_psi(u_bar::Function, value::Float64)
    return -u_bar(value)
end

function dy_psi_galerkin(size::Int64, u_bar_coefficients, value::Float64)
    return -u_galerkin(size, u_bar_coefficients, value)
end

function u_galerkin_coefficients(size::Int64, dq_coefficients, first_u_coefficient, du_bar::Function, N::Function, LMatrix, BMatrix)
    r_piece1 = BMatrix*dq_coefficients
    r_piece2 = dy_vartheta_bar(N,du_bar,1.0)*p_psi_function(size,1.0)
    r_piece3 = dy_vartheta_bar(N,du_bar,0.0)*p_psi_function(size,0.0)
    rhs = r_piece1-r_piece2+r_piece3
    lhs = LMatrix[2:size,2:size]
    rhs = rhs[2:size]
    coefficients = lhs \ rhs
    return prepend!(coefficients,[first_u_coefficient])
end

function u_galerkin(size::Int64, coefficients, value::Float64)
    sumTotal = 0
    for i=0:size-1
        sumTotal = sumTotal + coefficients[i+1]*(jacobi(2.*value -1., i, 0,0)+b_n(i)*jacobi(2.*value -1.,i+2,0,0))
    end
    return sumTotal
end

u_galerkin (generic function with 1 method)

In [4]:
function U_ij(size::Int64, u_galerkin_coefficients)
    fullArray = zeros(size,size)
    values, weights = gausslegendre(nodePoints)
    for i=0:size-1
        for j=0:size-1
            sumTotal = 0
            for k=0:nodePoints-1
                p_psi = jacobi(values[k+1],i,0,0)+b_n(i)*jacobi(values[k+1],i+2,0,0)
                p_q = jacobi(values[k+1],j,0,0)
                u_galerk = u_galerkin(size, u_galerkin_coefficients, (values[k+1]+1.)/2.)
                sumTotal = sumTotal + weights[k+1]*p_psi*p_q*u_galerk
            end
            fullArray[i+1, j+1] = .5*sumTotal
        end
    end
    return fullArray
end

function Q_ij(size::Int64, dq_galerkin_coefficients)
    fullArray = zeros(size,size)
    values, weights = gausslegendre(nodePoints)
    for i=0:size-1
        for j=0:size-1
            sumTotal = 0
            for k=0:nodePoints-1
                dq_galerk = dq_galerkin(size, dq_galerkin_coefficients, (values[k+1]+1.)/2.)
                p_psi_i = jacobi(values[k+1],i,0,0)+b_n(i)*jacobi(values[k+1],i+2,0,0)
                p_psi_j = jacobi(values[k+1],j,0,0)+b_n(j)*jacobi(values[k+1],j+2,0,0)
                sumTotal = sumTotal + weights[k+1]*dq_galerk*p_psi_i*p_psi_j
            end
            fullArray[i+1, j+1] = .5*sumTotal
        end
    end
    return fullArray
end

Q_ij (generic function with 1 method)

In [5]:
function N_constant(value::Float64)
    return 1.
end

function z_linear(value::Float64)
    return value
end

function dz(value::Float64)
    return 1
end 

function dq(value::Float64)
    return 0
end
first_coefficient = 0.5
matrixSize = 15
B = B_ij(matrixSize)
L = L_ij(matrixSize, N_constant)
M = M_ij(matrixSize)

dqc = dq_galerkin_coefficients(matrixSize, dq)
ugc = u_galerkin_coefficients(matrixSize, dqc, first_coefficient, dz, N_constant, L, B)

u_galerkin(matrixSize, ugc, 1.)
#U_ij(matixSize, ugc)

#dq_galerkin(matrixSize, dqc, .3)
#Q_ij(matrixSize, dqc)

0.9958333333333343

In [6]:
function psi(size, kx, ky, M_matrix, L_matrix, value)
    first_piece = inv((kx^2+ky^2)*M_matrix+L_matrix)
    second_piece = p_psi_function(size, value)
    return first_piece*second_piece
end

function growthRateEigenValues(sizeOfMatrix, beta_value, ky, kxValues, steps)
    B = B_ij(sizeOfMatrix)
    L = L_ij(sizeOfMatrix, N_constant)
    M = M_ij(sizeOfMatrix)
    dq_coefficients = dq_galerkin_coefficients(sizeOfMatrix, dq)
    u_coefficients = u_galerkin_coefficients(sizeOfMatrix, dq_coefficients, first_coefficient, dz, N_constant, L, B)
    U = U_ij(sizeOfMatrix, u_coefficients)
    Q = Q_ij(sizeOfMatrix, dq_coefficients)
    println("Matrices Computed")
    #Should have stable eigenvalues
    (values, vectors) = eigs(U, B)
    println(values)
    
    growthRates = zeros(steps)
    for i=1:steps
        #println()
        kx = kxValues[i]
        #println("k_x value $kx")
        dvt_plus = dy_vartheta_bar(N_constant, dz, 1.)
        dvt_minus = dy_vartheta_bar(N_constant, dz, 0.)
        ul = u_galerkin(sizeOfMatrix, u_coefficients, 1.)+dvt_plus*vecdot(p_psi_function(sizeOfMatrix, 1.),psi(sizeOfMatrix,kx,ky,M,L,1.))
        ur = -dvt_plus*vecdot(p_psi_function(sizeOfMatrix, 1.),psi(sizeOfMatrix,kx,ky,M,L,0.))
        ll = dvt_minus*vecdot(p_psi_function(sizeOfMatrix, 0.),psi(sizeOfMatrix,kx,ky,M,L,1.))
        lr = u_galerkin(sizeOfMatrix, u_coefficients, 0.)-vecdot(p_psi_function(sizeOfMatrix, 0.),psi(sizeOfMatrix,kx,ky,M,L,0.))
        lhs = [[ul ur]; [ll lr]]
        #println(lhs)
        eigenObject = eigfact(lhs)
        max_im = maximum(imag(eigenObject[:values]))
        growthRates[i] = max_im*kx
    end
    return growthRates
end

growthRateEigenValues (generic function with 1 method)

In [11]:
using Plots

nodePoints = 1000
matrixSize = 15
kxStart = 0.1
kxFinish = 2
steps = 50
beta = 0
ky = 0

kxs = linspace(kxStart, kxFinish, steps)
values = growthRateEigenValues(matrixSize, beta, ky, kxs, steps)
print(values)
plot(kxs, values)

Matrices Computed
Complex{Float64}[-11.9044+7.968im, -11.9044-7.968im, -4.58712+13.4585im, -4.58712-13.4585im, -9.99005+9.85338im, -9.99005-9.85338im]
[9.98325, 7.18251, 5.60205, 4.58558, 3.8758, 3.35119, 2.94687, 2.62503, 2.36213, 2.14275, 1.95635, 1.79549, 1.65471, 1.52997, 1.41816, 1.31684, 1.22408, 1.13828, 1.05814, 0.982514, 0.910403, 0.840883, 0.773062, 0.706019, 0.638727, 0.569924, 0.497873, 0.419809, 0.330336, 0.214163, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

LoadError: [91merror compiling _do_plot_show: error compiling _display: could not load library "libGR.dll"
The specified module could not be found.
[39m